In [1]:
import torch
import re
from grid.clients.torch import TorchClient

/Users/atrask/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
client = TorchClient(verbose=False)


UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: client:Qma5efEQtBUxDonmSDajv9dZ52axH9MNULKTgVpXWK8Ak2
all parts .... ['', 'Users', 'atrask', '.openmined']
full path /
full path /Users/
full path /Users/atrask/
full path /Users/atrask/.openmined/

UPDATE: Querying known workers...
	WORKER: /p2p-circuit/ipfs/QmXkWUybbTnfvFH8SUcrug6RGTLYTB23gSockKLxueR1vQ...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmXVoAQN2hHf5VF5SfCV4kQqZqNuZi3PE1so3pMYFguSUS...SUCCESS!!!


UPDATE: Searching for IPFS nodes - 9 found overall - 4 are OpenMined workers          

SUCCESS: Found 4 OpenMined nodes!!!

Hooking into Torch...
Overloading complete.


In [17]:
client.print_network_stats()

 ANCHOR - 1 - ID:N2obX  Ping:0.22sec  CPUs:1  CPU Load:5.0  Disk-util:98.3%  RAM-util:8.29%  GPUs:[]
 ANCHOR - 2 - ID:guSUS  Ping:0.16sec  CPUs:1  CPU Load:5.0  Disk-util:97.5%  RAM-util:20.2%  GPUs:[]
 ANCHOR - 3 - NAME:jm_anchor  Ping:0.21sec  CPUs:1  CPU Load:6.9  Disk-util:48.1%  RAM-util:53.3%  GPUs:[]
COMPUTE - 4 - NAME:hung  Ping:0.35sec  CPUs:4  CPU Load:1.7  Disk-util:16.5%  RAM-util:47.3%  GPUs:[]


In [20]:
compute_nodes = [x for x in client if re.match('compute:', x)]
for x in compute_nodes:
    print(x)

In [19]:
laptop = compute_nodes[0]
desktop = compute_nodes[1]

IndexError: list index out of range

In [59]:
# Convenient way to send specific tensors to certain nodes
assign_tensors_to_nodes = {}
assign_tensors_to_nodes[desktop] = []
assign_tensors_to_nodes[laptop] = []

In [60]:
# Loop over available nodes
for node in compute_nodes:
    for i in range(5):
        # Just getting five randomly generated 2x2 matrices here
        assign_tensors_to_nodes[node].append(torch.FloatTensor(2,2))
        
# Fill desktop tensors with 1s
for tensor in assign_tensors_to_nodes[desktop]:
    tensor.fill_(1)
    
# Fill desktop tensors with 2s
for tensor in assign_tensors_to_nodes[laptop]:
    tensor.fill_(2)

RecursionError: maximum recursion depth exceeded

In [16]:
# List of tensors -- tensors are local
# Sanity check -- we want desktop to have 2x2 matrices filled with 1s
assign_tensors_to_nodes[desktop]

[
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2]]

In [17]:
# List of tensors -- tensors are local
# Sanity check -- we want laptop to have 2x2 matrices filled with 2s
assign_tensors_to_nodes[laptop]

[
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2]]

In [18]:
# Multiply all tensors we want to send to desktop.
# Computation is done on this machine
accumulate = assign_tensors_to_nodes[desktop][0]
for x in assign_tensors_to_nodes[desktop][1:]:
    accumulate = accumulate.matmul(x)
print(accumulate)


 16  16
 16  16
[torch.FloatTensor of size 2x2]



In [19]:
# Multiply all tensors we want to send to laptop.
# Computation is done on this machine
accumulate = assign_tensors_to_nodes[laptop][0]
for x in assign_tensors_to_nodes[laptop][1:]:
    accumulate = accumulate.matmul(x)
print(accumulate)


 512  512
 512  512
[torch.FloatTensor of size 2x2]



In [20]:
# Send tensors to desktop node
for send_this_to_desktop in assign_tensors_to_nodes[desktop]:
    send_this_to_desktop.send_(desktop)

# Send tensors to laptop node
for send_this_to_laptop in assign_tensors_to_nodes[laptop]:
    send_this_to_laptop.send_(laptop)

In [21]:
# List of desktop tensors, now on desktop node
assign_tensors_to_nodes[desktop]

[[torch.FloatTensor - Locations:['compute:QmSdnVn6UTVnRj1AsMufjrMWkL3Ga1y91gwihD1spjrfSN']],
 [torch.FloatTensor - Locations:['compute:QmSdnVn6UTVnRj1AsMufjrMWkL3Ga1y91gwihD1spjrfSN']],
 [torch.FloatTensor - Locations:['compute:QmSdnVn6UTVnRj1AsMufjrMWkL3Ga1y91gwihD1spjrfSN']],
 [torch.FloatTensor - Locations:['compute:QmSdnVn6UTVnRj1AsMufjrMWkL3Ga1y91gwihD1spjrfSN']],
 [torch.FloatTensor - Locations:['compute:QmSdnVn6UTVnRj1AsMufjrMWkL3Ga1y91gwihD1spjrfSN']]]

In [22]:
# List of laptop tensors, now on laptop node
assign_tensors_to_nodes[laptop]

[[torch.FloatTensor - Locations:['compute:QmPWXB3APBhPJUZ2SDQhRxWpfFxsRSoC99roBivYKZAtTq']],
 [torch.FloatTensor - Locations:['compute:QmPWXB3APBhPJUZ2SDQhRxWpfFxsRSoC99roBivYKZAtTq']],
 [torch.FloatTensor - Locations:['compute:QmPWXB3APBhPJUZ2SDQhRxWpfFxsRSoC99roBivYKZAtTq']],
 [torch.FloatTensor - Locations:['compute:QmPWXB3APBhPJUZ2SDQhRxWpfFxsRSoC99roBivYKZAtTq']],
 [torch.FloatTensor - Locations:['compute:QmPWXB3APBhPJUZ2SDQhRxWpfFxsRSoC99roBivYKZAtTq']]]

In [23]:
# Multiply all tensors on desktop.
# Computation is done on desktop.
accumulate_desk = assign_tensors_to_nodes[desktop][0]
for x in assign_tensors_to_nodes[desktop][1:]:
    accumulate_desk = accumulate_desk.matmul(x)
print(accumulate_desk)

[torch.FloatTensor with no dimension]



In [24]:
# Multiply all tensors on laptop.
# Computation is done on laptop.
accumulate_lap = assign_tensors_to_nodes[laptop][0]
for x in assign_tensors_to_nodes[laptop][1:]:
    accumulate_lap = accumulate_lap.matmul(x)
print(accumulate_lap)

[torch.FloatTensor with no dimension]



In [25]:
# Returns the data resulting from desktop computation
accumulate_desk.get_()


 16  16
 16  16
[torch.FloatTensor of size 2x2]

In [26]:
# Returns the data resulting from laptop computation
accumulate_lap.get_()


 512  512
 512  512
[torch.FloatTensor of size 2x2]

In [27]:
(2 * accumulate_desk ** 2 == accumulate_lap).all()

True

In [28]:
(accumulate_lap == accumulate).all()

True